# Tier Wrapper generator

The following notebook generates the wrapping code to make C++ available to the pyclesperanto python code.
This notebook is to be runned for each Tiers (1,2,3, etc.) and will generate the corresponding wrapping code.
It only need to be runned when a Tier is updated.

__*WARNING*__: this can break the code if the python package is not update consequently. Please check the code and run the tests before pushing.

In [1]:
import glob
import os
import sys
import numpy as np

# define __file__ if it is not defined
if '__file__' not in globals():
    __file__ = os.path.abspath('generate-tiers-package.ipynb')

# get platform linux, windows, darwin
platform = sys.platform
# get platform architecture x86_64, x86, armv7l
architecture = os.uname().machine
# get the python version
python_version = sys.version_info

Get the list of hpp files in the tiers directory and sort them by name.

In [2]:
# get first argument of the script
tier = 3 #sys.argv[1]

# get current directory
current_dir = os.path.dirname(os.path.abspath(__file__))
# get the relative path to directory and add 'tier' plus the number of the tier
tier_dir = os.path.join(current_dir, '..', '_skbuild', 'linux-x86_64-3.10', 'cmake-build', '_deps', 'clic_lib-src', 'clic', 'include', 'tier' + str(tier))
# list all files in the tier_dir
files = glob.glob(tier_dir + '/*.hpp')
files.sort()
print(files)

['/mnt/data/Clesperanto/repo/pyclesperanto/scripts/../_skbuild/linux-x86_64-3.10/cmake-build/_deps/clic_lib-src/clic/include/tier3/cleCloseIndexGapsInLabelMapKernel.hpp', '/mnt/data/Clesperanto/repo/pyclesperanto/scripts/../_skbuild/linux-x86_64-3.10/cmake-build/_deps/clic_lib-src/clic/include/tier3/cleDifferenceOfGaussianKernel.hpp', '/mnt/data/Clesperanto/repo/pyclesperanto/scripts/../_skbuild/linux-x86_64-3.10/cmake-build/_deps/clic_lib-src/clic/include/tier3/cleHistogramKernel.hpp']


Define a set of function to parse the hpp files and extract the information we need.

In [3]:
def read_function_signature(file, function_name):
    with open(file, 'r') as f:
        # read the file content
        content = f.read()
        # find the position of the function of interest
        start = content.find(function_name + 'Kernel_Call')
        # find the position of the function signature
        start = content.find('(', start)
        # find the position of the end of the function signature
        end = content.find(')', start)
        # extract the function signature
        function_signature = content[start:end+1]
        # remove the prefix 'cle' from the function signature
        function_signature = function_signature.replace('cle', '')
        # remove the suffix 'Kernel_Call' from the function signature
        function_signature = function_signature.replace('Kernel_Call', '')
    return function_signature

def get_parameter_names(function_signature):
    # split the function signature into a list of parameters
    parameters = function_signature[1:-1].split(',')
    # get the parameter names
    parameter_names = [p.split(' ')[-1] for p in parameters]
    return parameter_names

def generate_wrapper_function(function_name, parameter_names):
    # generate the function name for python and c++
    py_function_name = '_' + function_name + 'Kernel_Call'
    cpp_function_name = function_name + 'Kernel_Call'
    # generate the function body for the wrapper
    wrapper_function = "m.def(\"" + py_function_name + "\", &cle::" + cpp_function_name + ", \"Call " + cpp_function_name + " from C++.\",\n"
    wrapper_function += "pybind11::arg(\""+ parameter_names[0] +"\")"
    for parameter_name in parameter_names[1:]:
        wrapper_function += ", pybind11::arg(\""
        wrapper_function += parameter_name
        wrapper_function += "\")"
    wrapper_function += ");"
    return wrapper_function


For each files, we extract the name of the function it contains as well as its signature. From both we build the wrapper code to make the function visible in Python.

In [4]:
wrapper_functions = []
# we loop on all the tier files
for file in files:
    # get the function name without the prefix 'cle' and the suffix 'Kernel_Call'
    function_name = os.path.basename(file)[3:-10]
    # print(function_name)

    # read the function signature from the file
    function_signature = read_function_signature(file, function_name)
    # print(function_signature)

    # get the parameter names
    parameter_names = get_parameter_names(function_signature)
    # print(parameter_names)

    # generate the wrapper function
    wrapper_function = generate_wrapper_function(function_name, parameter_names)
    # print(wrapper_function)

    # add the wrapper function to the list of wrapper functions
    wrapper_functions.append(wrapper_function)

# print each wrapper function on a new line 
for wrapper_function in wrapper_functions:
    print("\t",wrapper_function)

	 m.def("_CloseIndexGapsInLabelMapKernel_Call", &cle::CloseIndexGapsInLabelMapKernel_Call, "Call CloseIndexGapsInLabelMapKernel_Call from C++.",
pybind11::arg("device"), pybind11::arg("src"), pybind11::arg("dst"), pybind11::arg("value"));
	 m.def("_DifferenceOfGaussianKernel_Call", &cle::DifferenceOfGaussianKernel_Call, "Call DifferenceOfGaussianKernel_Call from C++.",
pybind11::arg("device"), pybind11::arg("src"), pybind11::arg("dst"), pybind11::arg("sigma1_x"), pybind11::arg("sigma1_y"), pybind11::arg("sigma1_z"), pybind11::arg("sigma2_x"), pybind11::arg("sigma2_y"), pybind11::arg("sigma2_z"));
	 m.def("_HistogramKernel_Call", &cle::HistogramKernel_Call, "Call HistogramKernel_Call from C++.",
pybind11::arg("device"), pybind11::arg("src"), pybind11::arg("dst"), pybind11::arg("min_value"), pybind11::arg("max_value"), pybind11::arg("bin"));


We create the wrapper file to contains the Tier functions currently processed. And for each function, we add the wrapper code to the file.

In [5]:
# create a cpp file for the current tier call cleTierX.cpp

# get the path to the current directory
current_dir = os.path.dirname(os.path.abspath(__file__))
# get the path to the directory where the cpp file should be stored
cpp_dir = os.path.join(current_dir, '..', 'wrapper')
# create the cpp file
cpp_file = os.path.join(cpp_dir, 'cleTier' + str(tier) + '.cpp')

# if the file already exists, rename it to cleTierX_old.cpp
if os.path.exists(cpp_file):
    os.rename(cpp_file, cpp_file.replace('.cpp', '_old.cpp'))

with open(cpp_file, 'w') as f:
    # write the header for pyclesperanto
    f.write('#include "pyclesperanto.hpp"\n')
    f.write('#include "cleKernelList.hpp"\n')
    # skip a line
    f.write('\n')
    # write the module function
    f.write('auto init_cleTier' + str(tier) + '(pybind11::module_ &m) -> void\n{\n')
    # skip a line
    f.write('\n')
    # write the wrapper functions
    for wrapper_function in wrapper_functions:
        f.write('\t' + wrapper_function + '\n')
        f.write('\n')
    # close the module function
    f.write('}')